In [14]:
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
import os
import pickle

In [15]:
with open(r'C:\Users\attic\HLS_Kelp\random_forest\rf', 'rb') as f:
    rf = pickle.load(f)
classified_path = r'C:\Users\attic\HLS_Kelp\imagery\rf_classified'
unclassified_path = r'C:\Users\attic\HLS_Kelp\imagery\rf_prepped'
files = os.listdir(unclassified_path)

In [17]:
for i, file in enumerate(files):
    if os.path.isfile(os.path.join(classified_path, f'{file}_kelp_classified.tif')):
        continue
    with rasterio.open(os.path.join(unclassified_path,file)) as img:
        file_img = img.read(indexes=[1, 2, 3, 4, 5, 6])
        width,height = img.shape
        img_data= file_img.reshape(file_img.shape[0], -1).T
        kelp_data = rf.predict(img_data)
        kelp_img = kelp_data.reshape(width, height)
        data_type = rasterio.uint8
        profile = {
            'driver': 'GTiff',
            'width': width,
            'height': height,
            'count': 1,  # one band
            'dtype': data_type,  # assuming binary mask, adjust dtype if needed
            'crs': img.crs,
            'transform': img.transform,
            'nodata': 0  # assuming no data is 0
        }
        # Write the land mask array to GeoTIFF
        with rasterio.open(os.path.join(classified_path, f'{file}_kelp_classified.tif'), 'w', **profile) as dst:
            dst.write(kelp_img.astype(data_type), 1)
        print(f'{i+1} / {len(files)}')
        

2 / 11
3 / 11


KeyboardInterrupt: 